https://dacon.io/competitions/official/236082/codeshare/7887?page=1&dtype=recent

In [1]:
# !pip install opencv-python
# !pip3 install torch torchvision torchaudio
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
# !pip install albumentations
# !python.exe -m pip install --upgrade pip

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
all_img_list = glob.glob('D:/Dacon_Data/2305_Papering/train/*/*')

In [7]:
for i in range(len(all_img_list)):
    all_img_list[i] = all_img_list[i].replace('\\', '/')

all_img_list

['D:/Dacon_Data/2305_Papering/train/가구수정/0.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/1.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/10.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/11.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/2.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/3.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/4.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/5.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/6.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/7.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/8.png',
 'D:/Dacon_Data/2305_Papering/train/가구수정/9.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/0.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/1.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/10.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/100.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/101.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/102.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/103.png',
 'D:/Dacon_Data/2305_Papering/train/걸레받이수정/1

In [8]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
# df['img_path'].replace('\\', '//')
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])

In [9]:
df

,img_path,label
0,D:/Dacon_Data/2305_Papering/train/가구수정/0.png,가구수정
1,D:/Dacon_Data/2305_Papering/train/가구수정/1.png,가구수정
2,D:/Dacon_Data/2305_Papering/train/가구수정/10.png,가구수정
3,D:/Dacon_Data/2305_Papering/train/가구수정/11.png,가구수정
4,D:/Dacon_Data/2305_Papering/train/가구수정/2.png,가구수정
...,...,...
3452,D:/Dacon_Data/2305_Papering/train/훼손/995.png,훼손
3453,D:/Dacon_Data/2305_Papering/train/훼손/996.png,훼손
3454,D:/Dacon_Data/2305_Papering/train/훼손/997.png,훼손
3455,D:/Dacon_Data/2305_Papering/train/훼손/998.png,훼손


In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [11]:
train

,img_path,label
3322,D:/Dacon_Data/2305_Papering/train/훼손/878.png,훼손
2089,D:/Dacon_Data/2305_Papering/train/훼손/1030.png,훼손
2321,D:/Dacon_Data/2305_Papering/train/훼손/124.png,훼손
2599,D:/Dacon_Data/2305_Papering/train/훼손/226.png,훼손
2305,D:/Dacon_Data/2305_Papering/train/훼손/1225.png,훼손
...,...,...
2945,D:/Dacon_Data/2305_Papering/train/훼손/538.png,훼손
1327,D:/Dacon_Data/2305_Papering/train/오염/365.png,오염
823,D:/Dacon_Data/2305_Papering/train/면불량/82.png,면불량
2767,D:/Dacon_Data/2305_Papering/train/훼손/378.png,훼손


### Label Encoding

In [12]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

### Custom Dataset

In [13]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [14]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
                                        max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
                                        max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [15]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### Model Define

In [16]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

### Train

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [18]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

### Run

In [19]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, 
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/76 [00:00<?, ?it/s]

TypeError: image must be numpy array type

### Inference

In [ ]:
test = pd.read_csv('D:/Dacon_Data/2305_Papering/test.csv')

In [ ]:
test

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

### Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)